## A) Descriptive analysis
1. Please describe the format of the data files. Can you identify any limitations or distortions of the data?
2. What is the most popular name of all time? (Of either gender.)
3. What is the most gender ambiguous name in 2013? 1945?
4. Of the names represented in the data, find the name that has had the largest percentage increase in popularity since 1980. Largest decrease?
5. Can you identify names that may have had an even larger increase or decrease in popularity?

### Part 1 - Understanding the file

- Data Format:

    - The data is provided in comma-delimited text files (.TXT), one for each state.

    - Each record contains five fields: State Code, Sex (M/F), Year of Birth, Name, and the number of occurrences.

- Data Limitations & Distortions:

    1. Exclusion of Uncommon Names: The most significant limitation is that names with fewer than 5 occurrences in a given state and year are completely omitted. This distorts the data by over-representing popular names and making it impossible to study the "long tail" of rare or unique names.

    2. Inaccurate National Aggregates: As a direct result of the first point, summing the state data will not produce an accurate national count. The true national total for any name will be higher than what can be calculated from these files.

    3. Source-Related Bias: The data is sourced exclusively from Social Security records. This means individuals born before the widespread adoption of Social Security in the mid-1930s, or those who never received a Social Security Number for other reasons, are not included. This particularly impacts the reliability of the data for the earliest years in the dataset.

### Part 2 - Searching through the data

In [1]:
#### Import necessary libraries
import os
import glob             # Finds all the files ending with .txt
import pandas as pd
import numpy as np
from dotenv import load_dotenv


In [2]:
# Load variables from the .env file
load_dotenv()

# Get the data path from the environment variables
path = os.getenv("DATA_PATH")

# Check if the path was loaded correctly
if not path:
    raise ValueError("DATA_PATH was not found!")

# Locate the files ending in .txt
all_files = glob.glob(path + "/*.TXT")

# Create an empty list for all the dataframes
df_list = []

# Provide the column names since the text files do not
column_names = ['State','Gender','Birth_year','Name','Name_occurrence']

# loop through all the text files and create a dataframe for each
# Add each dataframe to the df_list
for filename in all_files:
    df = pd.read_csv(filename, header=None, names=column_names)
    df_list.append(df)

# Add dataframes together along the row (axis=0)
# Show the dataframe
names_df = pd.concat(df_list, axis=0, ignore_index=True)
print("Data was successfully loaded!")
names_df.head()


Data was successfully loaded!


,State,Gender,Birth_year,Name,Name_occurrence
0,IN,F,1910,Mary,619
1,IN,F,1910,Helen,324
2,IN,F,1910,Ruth,238
3,IN,F,1910,Dorothy,215
4,IN,F,1910,Mildred,200


#### Part 3 - Boy or Girl

- To determine the most gender ambiguous name lets find the name with as much male and female matches

In [3]:
# Function to isolate the names by a specific year
def names_by_year(dataframe, column_name, year):
    """
    Filters a DataFrame to select rows for a specific year.

    Args:
        dataframe (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column containing the year.
        year (int): The year to filter by.

    Returns:
        pd.DataFrame: A new DataFrame containing only the data for the specified year.
    """
    names_in_year = dataframe[dataframe[column_name] == year]
    return names_in_year

In [4]:
names_2013 = names_by_year(names_df, 'Birth_year', 2013)
names_1945 = names_by_year(names_df, 'Birth_year', 1945)

In [7]:
# Function to calculate the total for each name in a specific year
def get_name_totals(dataframe_year, name_col, sex_col, count_col):
    """
    Groups a DataFrame by name and sex to calculate the total count for each name.

    Args:
        dataframe_year (pd.DataFrame): The input DataFrame (e.g., names_2013).
        name_col (str): The column name for the names.
        sex_col (str): The column name for the sex/gender.
        count_col (str): The column name for the counts/occurrences.

    Returns:
        pd.DataFrame: A new DataFrame with the summed counts for each name/sex combination.
    """
    # Group by the name and sex columns, then sum the count column for each group.
    name_totals = dataframe_year.groupby([name_col, sex_col])[count_col].sum().reset_index()
    return name_totals

In [9]:
names_total_2013 = get_name_totals(names_2013, 'Name','Gender', 'Name_occurrence')
names_total_1945 = get_name_totals(names_1945, 'Name','Gender', 'Name_occurrence')